In [2]:
import sys
import os

In [3]:
# check the installation #
if not 'HEADAS' in os.environ:
    raise RuntimeError('Heasoft is not initalized')

try:
    sys.path.insert(0, os.path.abspath(os.path.join(os.path.abspath(''), '..')))
    import heasoftpy as hsp
except ImportError:
    raise RuntimeError('Please ensure heasoftpy is in your PYTHONPATH')
    
if not hasattr(hsp, 'ftlist'):
    raise RuntimeError('Heasoftpy is not full installed. Please run the install.py script')

In [4]:
print(hsp.__version__)

1.2.1


In [5]:
from xspec import *
import numpy as np
import os
from scipy.optimize import fsolve

In [6]:
insts = ["FPMA",
        "FPMB",
        "pn",
        "m1",
        "m2"]
logNH = np.array([20,21,22,23,24,25])-22
logFs = [-12,-13,-13.5,-14]#np.linspace(-14,-13,3) #for 3-24 keV
print(logFs)
nh_gal = 0.0338 #average nH_gal in NEP sample
z = 0.5 #Mean redshift
exp = [360, #mean exposure across field
        360,
        249,
        249,
        249]

[-12, -13, -13.5, -14]


In [7]:
Fit.statMethod = 'cstat'

Default fit statistic is set to: C-Statistic
   This will apply to all current and newly loaded spectra.


In [10]:
def set_pars(m,pars,par_vals,
        par_fixed,one_epoch,
        par_var):
    """
    par_vals: the initial values for the parameters
    par_fixed: is the parameter fixed to the initial value?
    par_var: (if multiple_epochs are loaded) are pars linked across epochs?
    """
    n_eps = AllData.nGroups
    for i_par,par in enumerate(pars):
        #If the par is fixed, fix it
        if par_fixed[i_par]:
            m(par).values = [par_vals[i_par],-1]
        #else, if the par is free, free it
        else:
            m(par).values = par_vals[i_par]
            #THEN go through and unlink the parameters (if applicable)
            if not one_epoch and par_var[i_par]:
                for i_ep in range(1,n_eps+1):
                    #unlink the parameter, set it to the starter value
                    AllModels(i_ep)(par).values = par_vals[i_par]

In [16]:
def mk_clumpy(nh_gal,z,
        pars=[1,3,4,10,11,20,25],
        par_vals=[1.0,1,1.8,1.0e-4,1.0e-3,0.6,1.0e-4],
        par_fixed=[True,False,False,False,False,True,False],
        one_epoch=True,
        par_var=[False,False,False,False,False,False,False],
        bxa = False):
    """
    default pars: constant, nH, Gamma, norm, omni
    """
    Xset.abund="wilm" #Abundances for the uxclumpy model
    m = model.Model("constant*phabs(atable{uxclumpy-cutoff.fits} + constant*atable{uxclumpy-cutoff-omni.fits} + mekal)")
    m(2).values = [nh_gal,-1]#ISM absorption
    m(5).values = [400,-1] #E-cutoff
    m(6).values = [45,-1] #TORsigma
    m(7).values = [0.4,-1] #CTKcover
    m(8).values = [90.0,-1] #Theta_inc
    m(9).values = [z,-1,0.0,0.0,10.0,10.0] #Redshift
    m(11).values = [1.0e-5,1.0e-5,1.0e-5,1.0e-5,0.1,0.1]
    m(21).values = [1.0,-1] #nH for soft excess
    m(23).values = [z,-1] #redshift

    ######## omni component
    if AllData.nGroups == 0:
        n_spec = 1
    else:
        n_spec = AllData.nGroups
    for i_model in range(1,n_spec+1):
        AllModels(i_model)(12).link = m(3)
        AllModels(i_model)(13).link = m(4)
        AllModels(i_model)(14).link = m(5)
        AllModels(i_model)(15).link = m(6)
        AllModels(i_model)(16).link = m(7)
        AllModels(i_model)(17).link = m(8)
        AllModels(i_model)(18).link = m(9)
        AllModels(i_model)(19).link = m(10)
    set_pars(m, pars,par_vals,par_fixed,one_epoch,
        par_var)
    
    m(pars[0]).values = par_vals[0]
    m(1).values = 1.0,-1

In [17]:
mk_clumpy(nh_gal,z)

 Solar Abundance Vector set to wilm:  Wilms, J., Allen, A. & McCray, R. ApJ 542 914 (2000) (abundances are set to zero for those elements not included in the paper).

Model constant<1>*phabs<2>(atable{uxclumpy-cutoff.fits}<3> + constant<4>*atable{uxclumpy-cutoff-omni.fits}<5> + mekal<6>) Source No.: 1   Active/Off
Model Model Component  Parameter  Unit     Value
 par  comp
   1    1   constant   factor              1.00000      +/-  0.0          
   2    2   phabs      nH         10^22    1.00000      +/-  0.0          
   3    3   torus      nH                  10.0000      +/-  0.0          
   4    3   torus      PhoIndex            2.00000      +/-  0.0          
   5    3   torus      Ecut                100.000      +/-  0.0          
   6    3   torus      TORsigma            28.0000      +/-  0.0          
   7    3   torus      CTKcover            0.400000     +/-  0.0          
   8    3   torus      Theta_inc           18.2000      +/-  0.0          
   9    3   torus      z

In [18]:
m = AllModels(1)
m(25).values=1.0e-10
for logNH in logNH:
    m(3).values = 10.0**logNH
    for F in logFs:
        print("######################################################################")
        print("FLUX: {}".format(F))
        def flux_func(x, F = 10**float(F), i_norm=10):
            #Function for calculating the norm that yields a given flux
            if x < 0:
                x = 1.0e-10
            AllModels(1)(i_norm).values = x
            print(x)
            AllModels.calcFlux("3.0 24.0")
            flux = AllModels(1).flux[0]
            return flux-F
        int_norm = fsolve(flux_func,1.0e-5)[0]
        print("FLUX OF SPECTRUM: {}".format(AllModels(1).flux[0]))
        #Let’s start with 100 itterations
        for i in range(0,10):
            fk_s = [] #For fakeitsettings
            for i_i,inst in enumerate(insts): #FakeItSettings
                if os.path.exists("simulations/spec_files/{}_nH{}_F{}_{}s_{}.pha".format(inst,logNH+22,-F,exp[i_i],i)):
                    os.remove("simulations/spec_files/{}_nH{}_F{}_{}s_{}.pha".format(inst,logNH+22,-F,exp[i_i],i))
                fk_s.append(FakeitSettings(response="simulations/inst_files/{}.rmf".format(inst),
                    arf="simulations/inst_files/{}.arf".format(inst),
                    background="simulations/inst_files/{}_bgd.pha".format(inst),
                    exposure=exp[i_i]*1000,
                    fileName="simulations/spec_files/{}_nH{}_F{}_{}s_{}.pha".format(inst,logNH+22,-F,exp[i_i],i)))
            AllData.fakeit(len(insts),fk_s)
            #NuSTAR
            AllData(1).ignore("1.0e-60-3.0")
            AllData(1).ignore("24.0-200.0")
            AllData(2).ignore("1.0e-60-3.0")
            AllData(2).ignore("24.0-200.0")
            #XMM
            AllData(3).ignore("1.0e-60-0.1")
            AllData(3).ignore("6.0-200.0")
            AllData(4).ignore("1.0e-60-0.1")
            AllData(4).ignore("6.0-200.0")
            AllData(5).ignore("1.0e-60-0.1")
            AllData(5).ignore("6.0-200.0")
            if os.path.exists("simulations/fit_files/nH{}_F{}_{}s_{}_true.py".format(logNH+22,-F,exp[0],i)):
                os.remove("simulations/fit_files/nH{}_F{}_{}s_{}_true.py".format(logNH+22,-F,exp[0],i))
            print("Saving simulations/fit_files/nH{}_F{}_{}s_{}_true.py".format(logNH+22,-F,exp[0],i))
            Xset.save("simulations/fit_files/nH{}_F{}_{}s_{}_true.py".format(logNH+22,-F,exp[0],i))
            AllData.clear()

######################################################################
FLUX: -12
[1.e-05]
[1.e-05] Model Flux 1.7392e-06 photons (2.2113e-14 ergs/cm^2/s) range (3.0000 - 24.000 keV)

 Model Flux 1.7392e-06 photons (2.2113e-14 ergs/cm^2/s) range (3.0000 - 24.000 keV)
[1.e-05]
[1.00000001e-05]
 Model Flux 1.7392e-06 photons (2.2113e-14 ergs/cm^2/s) range (3.0000 - 24.000 keV)
 Model Flux 1.7392e-06 photons (2.2113e-14 ergs/cm^2/s) range (3.0000 - 24.000 keV)
[0.00066896]
[0.00045222] Model Flux 0.00011634 photons (1.4793e-12 ergs/cm^2/s) range (3.0000 - 24.000 keV)

[0.00045222] Model Flux 7.8648e-05 photons (1e-12 ergs/cm^2/s) range (3.0000 - 24.000 keV)

[0.00066896]
 Model Flux 7.8648e-05 photons (1e-12 ergs/cm^2/s) range (3.0000 - 24.000 keV)
[0.00045222]
[0.00034385] Model Flux 0.00011634 photons (1.4793e-12 ergs/cm^2/s) range (3.0000 - 24.000 keV)
 Model Flux 7.8648e-05 photons (1e-12 ergs/cm^2/s) range (3.0000 - 24.000 keV)

[0.00045222]
 Model Flux 5.9801e-05 photons (7.6036e-13 

/uufs/chpc.utah.edu/sys/installdir/r8/python/3.10.3/lib/python3.10/site-packages/scipy-1.8.0-py3.10-linux-x86_64.egg/scipy/optimize/_minpack_py.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


xsFakeit cmd string:
fakeit 1 simulations/inst_files/FPMA_bgd.pha 2 simulations/inst_files/FPMB_bgd.pha 3 simulations/inst_files/pn_bgd.pha 4 simulations/inst_files/m1_bgd.pha 5 simulations/inst_files/m2_bgd.pha & simulations/inst_files/FPMA.rmf & simulations/inst_files/FPMA.arf & simulations/inst_files/FPMB.rmf & simulations/inst_files/FPMB.arf & simulations/inst_files/pn.rmf & simulations/inst_files/pn.arf & simulations/inst_files/m1.rmf & simulations/inst_files/m1.arf & simulations/inst_files/m2.rmf & simulations/inst_files/m2.arf & y & / & simulations/spec_files/FPMA_nH20_F12_360s_0.pha & 360000, ,  & simulations/spec_files/FPMB_nH20_F12_360s_0.pha & 360000, ,  & simulations/spec_files/pn_nH20_F12_249s_0.pha & 249000, ,  & simulations/spec_files/m1_nH20_F12_249s_0.pha & 249000, ,  & simulations/spec_files/m2_nH20_F12_249s_0.pha & 249000, ,  

***Warning: Detected response matrix energy bin value = 0 (or neg).
     XSPEC will instead use small finite value (response file will not be

KeyboardInterrupt: 

In [24]:
import matplotlib.pyplot as plt

# Load your simulated data
s = Spectrum(["spec_files/FPMA_nH20_F12_360s_6_bkg.pha", "spec_files/FPMA_nH20_F12_360s_6.pha"])

s1 = Spectrum("file1")
b1 = s1.background
r1 = s1.response
arfFileName = r1.arf

# Define and fit a model (if needed)
#m = xspec.Model("your_model")
#m.fit(s)

# Plot the data and model
xspec.Plot.xAxis = "keV"  # Set the x-axis units
xspec.Plot("data")
xspec.Plot("model")

# Customize the plot using matplotlib
plt.xlabel("Energy (keV)")
plt.ylabel("Counts/s/keV")
plt.title("XSPEC Simulated Data")
plt.legend()
plt.show()

Exception: Filename parsing error